For training the model I'm using a table where all of the columns are different ailments, effects, and negatives, and each value in the dataframe are percentages corresponding to a specific rating for each. In order to compare input data from the user (which is just a list of things they are looking for in a strain) I needed to find a way to put it into a vector with zeros for the effects/negatives/ailments that they didn't specify, and a corresponding value for the ones that they did choose in the correct order.

In [ ]:
    # get lists of the stuff that's in the json object being passed
    # as the request 

    desired_dict = json.loads(request)
    effects, negatives, ailments = (
        desired_dict.get("effects"), 
        desired_dict.get("negatives"),
        desired_dict.get("ailments")
    )
    
    # this makes sure everything is lowercase cuz who knows 
    # how people are going to input

    effects = [effect.lower() for effect in effects]
    negatives = [negative.lower() for negative in negatives]
    ailments = [ailment.lower() for ailment in ailments]
    
    
    for index, effect in enumerate(effects):
        if effect in columns:
            
            # changes value to the index of the column in the database
            
            effects[index] = columns.index(effect)

    for index, negative in enumerate(negatives):
        if negative in columns:
            negatives[index] = columns.index(negative)

    for index, ailment in enumerate(ailments):
        if ailment in columns:
            ailments[index] = columns.index(ailment)

    # makes empty vector
            
    vector = [
        0 for _ in range(len(columns))
    ]    
    
    # starting weight to be assigned to first value
    
    weight = 100

    # now that each value in effects is assigned to an 
    # index:
    
    for index in effects:
        
        # check to make sure it is an integer
        
        if isinstance(index, int):
            
            # assign value to vector depending on
            # index
            
            vector[index] = weight
            weight *= .8
            weight = int(weight)

    weight = 100

    for index in negatives:
        if isinstance(index, int):
            vector[index] = weight
            weight *= .8
            weight = int(weight)
    
    weight = 100

    for index in ailments:
        if isinstance(index, int):
            vector[index] = weight
            weight *= .8
            weight = int(weight)

    # NOW it can be passed in as an actual vector (like how the data is set up during training)        
    
    data = numpy.array(vector)
    request_series = pd.Series(data,index=columns)
    distance, neighbors = nn.kneighbors([request_series])
    
#     list_strains = []
#     for points in neighbors:
#         for index in points:
#             list_strains.append(index)

#     return list_strains[:n]
            
# #     recommended = strains_df.iloc[list_strains].head(n)
# #     result = {
# #         #"errors": ", ".join(error_messages),
# #         "strains": recommended.to_dict("records")
# #     }
    
# #     return result

# # #json.dumps(result)